In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import dispatch_model
from accelerate.utils import infer_auto_device_map
import accelerate

model_name = "model/Qwen2.5-Coder-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.89s/it]


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
write a quick sort algorithm.<|im_end|>
<|im_start|>assistant



In [11]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(model_inputs)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   4934,    264,   3974,
           3378,  12111,     13, 151645,    198, 151644,  77091,    198]],
       device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}


In [16]:

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)


tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   4934,    264,   3974,
           3378,  12111,     13, 151645,    198, 151644,  77091,    198,  95456,
              0,  21193,    374,    264,  13027,   8129,    315,    279,  17251,
          15967,  12111,   1447,  73594,  12669,    198,    750,   3974,  18435,
          10939,    982,    262,    421,   2422,  10939,      8,   2651,    220,
             16,    510,    286,    470,   2890,    198,    262,    770,    510,
            286,  26045,    284,   2890,  24693,  10939,      8,    442,    220,
             17,    921,    286,   2115,    284,    508,     87,    369,    856,
            304,   2890,    421,    856,    366,  26045,    921,    286,   6149,
            284,    508,     87,    369,    856,    304,   2890,    421,    856,
            621,  26045,    

In [ ]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)